In [10]:
from keras.models import load_model
from keras.preprocessing.image import load_img, img_to_array
import os, shutil
import numpy as np
from skimage.transform import resize
#This program reads models from a folder (example: Those saved by Modelcheckpoint in tensorboard) and runs predictions on a
#folder(s) of images. Output displays the # of results from the cutoff to 1 individually and the % of the results at that cutoff 

#Variables to edit
imageheight=200
imagewidth=300
cutoff=0.7 #Above which is an echolcation and below is abnormal file-Used for displaying the min. cutoff to calculate from
mfolder = "./weights/"
#mfolder = "./models/" #Folder with saved models in h5 format
ifolder = "../abcopy/a/" #Folder with images to test model with
foldercheck2 = True #Do you want to check the 2nd folder of images?
ifolder2 = "../images/train/echolocation/" #Folder2 with images to test model with
foldercheck3 = False #Do you want to check the 3rd folder of images?
ifolder3 = "../abcopy/a/" #Folder3 with images to test model with
mincheck = 20 #Min starting model which represents the epoch that it was created-Models below this will be deleted from folder
#End of variables to edit

modcount=0 #Counts the model number for display 
def Testcutoffs(folder, filelist):
    abcount = [0] * cutoffdiff 
    for inum in range(len(filelist)):
        image = load_img(folder + filelist[inum], target_size=(imageheight,imagewidth))
        imageresize = image.resize((imagewidth, imageheight))#.convert('RGB') #Resize image for desired CNN model
        imagearray = img_to_array(imageresize) #Make image into an array
        imagearray = np.expand_dims(imagearray, 0) #Add 4th dimension to array for CNN -> will get error otherwise
        imagearray = imagearray.astype('float32')/255 #Convert array so that its predictions are readable
        prediction=model.predict(imagearray) #Run prediction on image-Returns list of float predictions
        for abdetermine in range(cutoffdiff):
            if prediction[0]<(cutoff+(abdetermine*0.1)):
                abcount[abdetermine]=abcount[abdetermine]+1
    
    for abprint in range(cutoffdiff):
        print('Abnormal @ cutoff {0:2.1f}: {1:2d} at {2:4.2f}% Echolocation count: {3:2d} at {4:4.2f}%'.format(cutoff+(abprint*0.1),
        abcount[abprint],abcount[abprint]*100/len(filelist),(len(filelist)-abcount[abprint]),             
        (len(filelist)-abcount[abprint])*100/len(filelist)))
        

print('Deleting models from 1 to {0:2d}'.format(mincheck))
for mdl in range(mincheck+1): #Remove unwanted models-Range value represents the last model-1 to delete
    if os.path.exists(mfolder+str(mdl)+".h5"):
        os.remove(mfolder+str(mdl)+".h5")
    if os.path.exists(mfolder+"0"+str(mdl)+".h5"):
        os.remove(mfolder+"0"+str(mdl)+".h5")
for ipremove in mfilelist: #Remove .ipynb_checkpoints folder which may get saved in models folder
    if '{}'.format(ipremove[0:6]) == '.ipynb':
        shutil.rmtree(mfolder+ipremove)
#Reading directories        
mfilelist = os.listdir(mfolder) #Make list of model filenames
ifilelist = os.listdir(ifolder) #Make list of image filenames
if foldercheck2 == True:
    ifilelist2 = os.listdir(ifolder2) #Make list of image filenames for 2nd scan
if foldercheck3 == True:
    ifilelist3 = os.listdir(ifolder3) #Make list of image filenames for 3rd scan

cutoffdiff = int((1-cutoff)*10) #Calculates the difference between the cutoff and 1-Used for calculating & displaying accuracies
for mdl in range(len(mfilelist)):
    modcount += 1
    print('{0:2d}. of {1:2d} Models: {2:s}'.format(modcount,len(mfilelist),mfilelist[mdl]))
    model = load_model(mfolder+mfilelist[mdl])
    print('Processing first folder with {0:2d} images'.format(len(ifilelist)))
    Testcutoffs(ifolder,ifilelist)
    if foldercheck2 == True:
        print('Processing second folder with {0:2d} images'.format(len(ifilelist2)))
        Testcutoffs(ifolder2,ifilelist2)
    if foldercheck3 == True:
        print('Processing third folder with {0:2d} images'.format(len(ifilelist3)))
        Testcutoffs(ifolder3,ifilelist3)
        
    print('End of model {0:s} predictions-----------------------'.format(mfilelist[mdl]))
print('Finished')

Deleting models from 1 to 20
 1. of  4 Models: 284.h5
Processing first folder with 37 images
Abnormal @ cutoff 0.7: 27 at 72.00% Echolocation count: 10 at 27.00%
Abnormal @ cutoff 0.8: 28 at 75.00% Echolocation count:  9 at 24.00%
Abnormal @ cutoff 0.9: 34 at 91.00% Echolocation count:  3 at 8.00%
Processing second folder with 3858 images
Abnormal @ cutoff 0.7:  8 at 0.00% Echolocation count: 3850 at 99.00%
Abnormal @ cutoff 0.8: 23 at 0.00% Echolocation count: 3835 at 99.00%
Abnormal @ cutoff 0.9: 80 at 2.00% Echolocation count: 3778 at 97.00%
End of model 284.h5 predictions-----------------------
 2. of  4 Models: 408filter900Sgd.h5
Processing first folder with 37 images
Abnormal @ cutoff 0.7: 27 at 72.00% Echolocation count: 10 at 27.00%
Abnormal @ cutoff 0.8: 32 at 86.00% Echolocation count:  5 at 13.00%
Abnormal @ cutoff 0.9: 33 at 89.00% Echolocation count:  4 at 10.00%
Processing second folder with 3858 images
Abnormal @ cutoff 0.7:  1 at 0.00% Echolocation count: 3857 at 99.00%